In [30]:
# Download excerpt of human chromosome 1
import wget
url = 'http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta'
filename = wget.download(url)

# Download Boyer-Moore preprocessing file
url = 'http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py'
filename2 = wget.download(url)

First, let's implement the Boyer-Moore matching with counts:

In [31]:
import bm_preproc
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = bm_preproc.BoyerMoore(p, lowercase_alphabet)

In [47]:
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching with count """
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                num_alignments += 1
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
            num_alignments += 1
        i += shift
    return occurrences, num_alignments, num_character_comparisons

In [48]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = bm_preproc.BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [49]:
p = 'needle'
t = 'needle need noodle needle'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = bm_preproc.BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


Next, the naive exact matching with counts:

In [53]:
def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            num_character_comparisons += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                num_alignments += 1
                break
        if match:
            occurrences.append(i)  # all chars matched; record
            num_alignments += 1
    return occurrences, num_alignments, num_character_comparisons

In [54]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [55]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


Now that we have our two working matching functions, we can start on the actual assingment

In [56]:
# Read in the downloaded genome
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readGenome('./chr1.GRCh38.excerpt.fasta')

**Question 1**
How many alignments does the naive exact matching algorithm try when matching the string *GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG* (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [57]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
naive_with_counts(p,genome)

([56922], 799954, 984143)

**Question 2**  
How many character comparisons does the naive exact matching algorithm try when matching the string *GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG* (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [58]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
naive_with_counts(p,genome)

([56922], 799954, 984143)

**Question 3**  
How many alignments does Boyer-Moore try when matching the string *GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG* (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [59]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
lowercase_alphabet = 'ATGC '
p_bm = bm_preproc.BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, genome)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


**Question 4**
Index-assisted approximate matching. In practicals, we built a Python class called **Index**

implementing an ordered-list version of the k-mer index.

In [139]:
import kmer_index
t="lets test out the kmer class function"
eightmer = kmer_index.Index(t, k = 8)
p = "class functiin"
eightmer.query(p)


[23]

In [145]:
t="lets test out the kmer class function"
n_mer_index = kmer_index.Index(t, k = 8)
p = "class functiin"
mm = 2

#p[0], n_mer_index.index[1][0][0]
n_mer_index.index

[(' class f', 22),
 (' functio', 28),
 (' kmer cl', 17),
 (' out the', 9),
 (' test ou', 4),
 (' the kme', 13),
 ('ass func', 25),
 ('class fu', 23),
 ('e kmer c', 16),
 ('er class', 20),
 ('est out ', 6),
 ('ets test', 1),
 ('function', 29),
 ('he kmer ', 15),
 ('kmer cla', 18),
 ('lass fun', 24),
 ('lets tes', 0),
 ('mer clas', 19),
 ('out the ', 10),
 ('r class ', 21),
 ('s functi', 27),
 ('s test o', 3),
 ('ss funct', 26),
 ('st out t', 7),
 ('t out th', 8),
 ('t the km', 12),
 ('test out', 5),
 ('the kmer', 14),
 ('ts test ', 2),
 ('ut the k', 11)]

In [149]:
t="lets test out the kmer class function"
n_mer_index = kmer_index.Index(t, k = 8)
p = "class functiin"
mm = 2

def naive_2mm(p, n_mer_index):
    occurrences = []
    # switch so the pattern is the text
    # and the kmer index of the text is the pattern
    # This is needed when len(p) > k
    t = p
    for index in n_mer_index.index:
        p = index[0]
        print(p)
        for i in range(len(t) - len(p) + 1):  # loop over alignments
            match = True
            mismatch = 0
            for j in range(len(p)):  # loop over characters
                if t[i+j] != p[j]:
                    mismatch += 1
                    if mismatch > 2:
                        match = False
                        break
            if match:
                occurrences.append(index[1])  # all chars matched; record
        return occurrences

naive_2mm(p,n_mer_index), n_mer_index.query(p)

 class f


([], [23])

In [150]:
for index in n_mer_index.index:
    print(index[0])
    len(t) - len(p) + 1



 class f
 functio
 kmer cl
 out the
 test ou
 the kme
ass func
class fu
e kmer c
er class
est out 
ets test
function
he kmer 
kmer cla
lass fun
lets tes
mer clas
out the 
r class 
s functi
s test o
ss funct
st out t
t out th
t the km
test out
the kmer
ts test 
ut the k


Double check with naive 2 mismatch matching

In [112]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mismatch = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:
                mismatch += 1
                if mismatch > 2:
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [114]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
len(naive_2mm(p,genome))

19